# Varying the second characteristic decay time and noise level in a viscoelastic model based on a Kelvin model

In [1]:
import os
import sys
import time
from datetime import datetime
import numpy as np
import torch

sys.path.append('../src')
import deepymod_torch.VE_datagen as VE_datagen
import deepymod_torch.VE_params as VE_params
from deepymod_torch.DeepMod import DeepMoD
from deepymod_torch.library_function import mech_library

/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

Here, we set many parameters that define our problem. We define:
- The shape, frequency and magnitude of the sinc input manipulation
- The model and model type (which in the case that input_type is stress is a Kelvin type)
- The sampling rate from the data that will be generated

In [2]:
# omega = 1 # Today, this is the parameter to vary
E = [1, 1, 1]
eta = [2.5, 0.5]
Amp = 7
# input_expr = lambda t: Amp*np.sin(omega*t)/(omega*t)
# d_input_expr = lambda t: (Amp/t)*(np.cos(omega*t) - np.sin(omega*t)/(omega*t))
# input_torch_lambda = lambda t: Amp*torch.sin(omega*t)/(omega*t)
input_type = 'Stress'
func_desc = 'Sinc'
number_of_samples = 1000
investigated_param = 'Varying manipulation omega'

To generate data, we need to choose where and when to evaluate the target data fr a given manipulation. Below, we choose those time points.

In [3]:
# time_array = np.linspace(10**-10, 10*np.pi/omega, 5000).reshape(-1, 1)

Next we configure DeepMoD. We configure:
- The initial L1 regularisation penalty
- The number of epochs at each stage of training
- The size and shape of the network
- The library function for calculating potential terms relevant to VE problems.

In [4]:
optim_config = {'lambda': 10**-6, 'lr_coeffs': 0.002, 'max_iterations': 2, 'mse_only_iterations': 2, 'final_run_iterations': 2}
network_config = {'input_dim': 1, 'hidden_dim': 30, 'layers': 4, 'output_dim': 1}
library_config = {'type': mech_library, 'diff_order': 3, 'coeff_sign': 'positive', 'input_type': input_type}#, 'input_expr': input_torch_lambda}

Here we define the decay constant values we wish to test, in this case those between $10^{-2}$ and $10^3$, doubling each step.

In [5]:
# tau_2_values = [0.01]
# while tau_2_values[-1] < 1000:
#     tau_2_values += [tau_2_values[-1]*2]

# tau_2_values

In [6]:
# omega_values = np.arange(5, 7.5, 0.2)
omega_values = np.arange(5, 5.3, 0.2)
omega_values

array([5. , 5.2])

Here we define the noise levels that we are going to test. In each case the noise level refers to the percentage of the standard deviation on the whole data set used to scale a noise adjustment that would otherwise be of the magnitude of a standard normal distribution.

In [7]:
noise_level = 0
# noise_level_values = [0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5]

In addition, each combination of $\tau_2$ and noise level will be tested 5 times, each time with a different random seed.

In [8]:
# number_of_seeds = 5

Next we run the loop which will generate the data on a tau_2 by tau_2 individual basis for each value above, as well as testing each of these tau_2 values at all noise levels of interest as specified above.

With this data, in each iteration of the loop:

- The data will be prepared for DeepMoD injection
- DeepMoD will try its best
- The results will be organised and saved in a named folder.
- The progress will be available in Tensorboard files also which will need to be manually dragged across after the loop is done (or during!)

The total number of tests are now:

In [9]:
# len(tau_2_values)*len(noise_level_values)*number_of_seeds

As this particular test doesn't change the model each test, the expected coefficients are the same each time, and can be calculated outside the loop, so that they are not recalculated every loop.

The coeffs DeepMoD should find vary a little however due to small variations in teh scaling performed each time.

In [10]:
if input_type == 'Stress':
    unscaled_expected_coeffs = VE_params.coeffs_from_model_params_kelvin(E, eta)
elif input_type == 'Strain':
    unscaled_expected_coeffs = VE_params.coeffs_from_model_params_maxwell(E, eta)

unscaled_expected_coeffs

[0.333333333333333,
 0.416666666666667,
 1.00000000000000,
 2.00000000000000,
 0.416666666666667]

Information for saving that doesn't change each loop....

In [11]:
parent_folder = '../data/Results'
first_subfolder = investigated_param.replace('.', '-')

Main loop!

In [12]:
for omega in omega_values:
    
    print('Generating data for omega value:', omega)
    
    # reset randomised initialisation
    np.random.seed(0)
    torch.manual_seed(0)

    # DATA GENERATION
    # redefine manipulation profile per new omega
    input_expr = lambda t: Amp*np.sin(omega*t)/(omega*t)
    d_input_expr = lambda t: (Amp/t)*(np.cos(omega*t) - np.sin(omega*t)/(omega*t))
    input_torch_lambda = lambda t: Amp*torch.sin(omega*t)/(omega*t)
    
    # Generate different length time series to correspond to same oscillations with new omega
    time_array = np.linspace(10**-10, 10*np.pi/omega, 5000).reshape(-1, 1)

    # generate data
    strain_array, stress_array = VE_datagen.calculate_strain_stress(input_type, time_array, input_expr, E, eta, D_input_lambda=d_input_expr)
    
    # Scale data (y scaling only as testing x scaling)
    strain_sf = 1/np.max(abs(strain_array))
    stress_sf = 1/np.max(abs(stress_array))
    if input_type == 'Strain':
        scaled_input_torch_lambda = lambda t: strain_sf*input_torch_lambda(t)
        scaled_target_array = stress_array*stress_sf
    elif input_type == 'Stress':
        scaled_input_torch_lambda = lambda t: stress_sf*input_torch_lambda(t)
        scaled_target_array = strain_array*strain_sf
    
    # add noise to value at each time point
#     noisy_strain_array = strain_array + noise_level * np.std(strain_array) * np.random.standard_normal(strain_array.shape)

    # randomly sample
    reordered_row_indices = np.random.permutation(time_array.size)
    reduced_time_array = time_array[reordered_row_indices, :][:number_of_samples]
    reduced_target_array = scaled_target_array[reordered_row_indices, :][:number_of_samples]


    # DEEPMOD PREPARATION
    # convert to tensors
    time_tensor = torch.tensor(reduced_time_array, dtype=torch.float32, requires_grad=True)
    target_tensor = torch.tensor(reduced_target_array, dtype=torch.float32)
    
    # load redefined torch expression for input based on change in omega into config
    library_config['input_expr'] = scaled_input_torch_lambda

    
    # DEEPMOD
    # record start time for later transfer of tensorboard files to correct folders
    now = datetime.now()
    dt_string = now.strftime('%d/%m/%Y %H:%M:%S')

    # run DeepMoD
    print('Running DeepMoD')
    sparse_coeff_vector_list_list, scaled_coeff_vector_list_list, sparsity_mask_list_list, network = DeepMoD(time_tensor, target_tensor, network_config, library_config, optim_config)
    print('Saving results')


    # ORGANISING RESULTS
    # calculate expected coeffs. Depending on input type, a different model will have been used to intepret the provided model parameters during data generation.
    # the choice of model must be taken into account to calculate the coeffs that are correct for the data generated.
#     if input_type == 'Stress':
#         expected_coeffs = VE_params.coeffs_from_model_params_kelvin(E, eta)
#     elif input_type == 'Strain':
#         expected_coeffs = VE_params.coeffs_from_model_params_maxwell(E, eta)

    # reshape, sample and convert to arrays all series data for homogeneity of form and saving
#     stress_array = stress_array.reshape(-1,1)
#     reduced_stress_array = stress_array[reordered_row_indices, :][:number_of_samples]
    prediction_array = np.array(network(time_tensor).detach().cpu())

    # Scale true coeffs to ones we expect to be found after scaling.
    # Convert list of expected coeffs to array for saving
    scaled_expected_coeffs = VE_params.scaled_coeffs_from_true(unscaled_expected_coeffs, 1, strain_sf, stress_sf)
    target_coeffs_array = np.array(scaled_expected_coeffs).reshape(-1,1)

    # convert pre-thresholding coeffs data to arrays for saving
    pre_thresh_coeffs_array = np.array(sparse_coeff_vector_list_list[0][0].detach().cpu())
    pre_thresh_scaled_coeffs_array = np.array(scaled_coeff_vector_list_list[0][0].detach().cpu())

    # convert final coeffs data to arrays for saving
    final_coeffs_array = np.array(sparse_coeff_vector_list_list[-1][0].detach().cpu())
    final_scaled_coeffs_array = np.array(scaled_coeff_vector_list_list[-1][0].detach().cpu())
    sparsity_mask_array = np.array(sparsity_mask_list_list[-1][0].cpu()).reshape(-1,1)

    # group like data vectors together for saving
    dg_series_data = np.concatenate((time_array, strain_array, stress_array), axis=1)
    NN_series_data = np.concatenate((reduced_time_array, reduced_target_array, prediction_array), axis=1)
    pre_thresh_coeffs_data = np.concatenate((pre_thresh_coeffs_array, pre_thresh_scaled_coeffs_array), axis=1)
    final_coeffs_data = np.concatenate((final_coeffs_array, final_scaled_coeffs_array, sparsity_mask_array), axis=1)
    
    # remove library from dictionary as we don't want to save that
    input_theta = library_config.pop('input_theta')
    
    # Gather miscellaneous information into lists for saving
    dg_info_list = ['E: '+str(E), 'eta: '+str(eta), 'Input: '+input_type, 'Desc: '+func_desc, 'omega: '+str(omega), 'Amp: '+str(Amp)]
    treatment_info_list = ['noise_factor: '+str(noise_level), 'time_sf: '+'1', 'strain_sf: '+str(strain_sf), 'stress_sf: '+str(stress_sf)]
    config_dict_list = ['optim: '+str(optim_config), 'network: '+str(network_config), 'library: '+str(library_config)]
    misc_list = ['date_stamp: '+dt_string]


    # SAVING RESULTS
    # collect parameters to name folder for saving
    second_subfolder = 'param_' + str(omega).replace('.', '-')
#     third_subfolder = 'noise_' + str(noise_level).replace('.', '-')
#     fourth_subfolder = 'seed_' + str(seed_value)
    foldername = parent_folder + '/' + first_subfolder + '/' + second_subfolder# + '/' + third_subfolder + '/' + fourth_subfolder

    # make folder
    if not os.path.isdir(foldername):
        os.makedirs(foldername)

    # save all array data
    np.savetxt(foldername+'/DG_series_data.csv', dg_series_data, delimiter=',', header='Time, Strain, Stress')
    np.savetxt(foldername+'/NN_series_data.csv', NN_series_data, delimiter=',', header='Time, Target, Prediction')
    np.savetxt(foldername+'/expected_coeffs.csv', target_coeffs_array, delimiter=',', header='Expected_coeffs')
    np.savetxt(foldername+'/pre_thresh_coeffs_data.csv', pre_thresh_coeffs_data, delimiter=',', header='Trained_Coeffs, Scaled_Trained_Coeffs')
    np.savetxt(foldername+'/final_coeffs_data.csv', final_coeffs_data, delimiter=',', header='Trained_Coeffs, Scaled_Trained_Coeffs, Sparsity_Mask')
    
    # save all lists data
    with open(foldername+'/DG_info_list.txt', 'w') as file:
        file.writelines("%s\n" % line for line in dg_info_list)
        
    with open(foldername+'/treatment_info_list.txt', 'w') as file:
        file.writelines("%s\n" % line for line in treatment_info_list)
    
    with open(foldername+'/config_dict_list.txt', 'w') as file:
        file.writelines("%s\n" % line for line in config_dict_list)
    
    with open(foldername+'/misc_list.txt', 'w') as file:
        file.writelines("%s\n" % line for line in misc_list)
        
    print('sleeping...')
    time.sleep(10)

Epoch | Total loss | MSE | PI | L1 
0 5.8E+01 6.9E-02 5.8E+01 0.0E+00
tensor([[0.1597],
        [1.6958],
        [0.1607],
        [0.7869],
        [1.2044],
        [0.2287],
        [0.7270]], requires_grad=True)
Time elapsed: 0.0 minutes 0.04239916801452637 seconds
Saving results
sleeping...
